In [1]:
import asyncio
from collections import Counter
import datetime
from functools import partial
import os
from pathlib import Path
import random
from typing import List
import uuid

from IPython.display import display, clear_output
import ipywidgets as widgets
from markdown import markdown
import MDAnalysis as mda
import nglview as nv

from database_utils import Base, User, Answer
from database_utils import create_db_engine_and_session

In [2]:
#The current notebook should be
#in the main "molecular similarity survey" directory.
dir_survey = Path.cwd()

In [3]:
dir_pics = Path(dir_survey, "images_2D")

bname_pics = "image_molecule"

In [4]:
dir_confs = Path(dir_survey, "conformers_3D")

bname_confs = "best_rocs_conformer"

In [5]:
dir_info = Path(dir_survey, "info_quiz")

In [6]:
loading = widgets.Image()
loading.set_value_from_file("info_quiz/loading.gif")
display(loading)

Image(value=b'GIF89a\x96\x00\x96\x00\xf7\xff\x00\x01\x01\x01\x1b\x1b\x1b444UUU\x81\x81\x81\x97\x97\x97\xb4\xb4…

In [7]:
dir_captcha = Path(dir_survey, "captcha")

In [8]:
#IDs of first and last molecular pair (considering all available pairs).
id_first_pair = 1
id_last_pair = 100
ids_all_pairs = list(range(id_first_pair, id_last_pair + 1))

In [9]:
suffixes_mols = ["a", "b"]

In [10]:
#Get a random UUID for each user.
#(see this: https://pynative.com/python-uuid-module-to-generate-universally-unique-identifiers/)
#We will use the hexadecimal string of the UUID as seed for the random number generator.
#If needed, the original UUID object can be easily recreated from the hexadecimal string.
id_user = uuid.uuid4().hex

In [11]:
#Number of pairs that will be randomly selected and presented to each user.
n_pairs = 5

#Use the UUID hexadecimal string as seed for random number generator,
#and select IDs of unique molecular pairs that will be presented to each user.
random.seed(id_user)
ids_pairs = random.sample(ids_all_pairs, n_pairs)

#Column that will contain the randomly selected pair IDs for each user.
colname_id_pair = "id_surveyPair"

In [12]:
#Dictionary that will store various objects associated with the database,
#so that they can be created and accessed by functions in the async task.
#This is a slightly better approach than to define global variables.
db_objects = {}

In [13]:
#Get URL that points to database that will be used by the app
#to save the answers. This app is supposed to run on Heroku,
#where the URL is stored in the following environment variable.
#If running the app locally, add the environment variable
#to the `.bashrc` file,
#pointing to a database of your choice.
db_objects["url"] = os.environ.get("DATABASE_URL")
if not db_objects["url"]:
    raise ValueError(f"No environment variable for database")
#SQLAlchemy >= 1.4 expects "postgresql", not "postgres", in database URL,
#but Heroku still uses "postgres".
#See: https://stackoverflow.com/questions/66690321/
db_objects["url"] = db_objects["url"].replace("postgres:", "postgresql:")

In [14]:
user = User(id=id_user,
            date=datetime.date.today(),
            )

In [15]:
def make_markdown_widget(file_md: Path, **kwargs_widget) -> widgets.HTML:
    """Create an HTML widget displaying Markdown text from a given Markdown file."""
    if not file_md.is_file():
        raise FileNotFoundError(f"Missing `{file_md}`.")
    
    widget_md = widgets.HTML(markdown(file_md.read_text()), **kwargs_widget)
    return widget_md

In [16]:
#Select a captcha picture, and get the correct captcha value.
random.seed(id_user)
file_captcha = random.choice(list(dir_captcha.glob("*.png")))
value_captcha = file_captcha.stem

image_captcha = widgets.Image().from_file(file_captcha)

label_captcha = widgets.Label("Please insert the text displayed in the image, then Submit")

field_captcha = widgets.Text()

submit_captcha = widgets.Button(description="Submit")

feedback_captcha = widgets.Output()


grid_captcha = widgets.GridBox([image_captcha,
                                label_captcha,
                                field_captcha,
                                submit_captcha,
                                feedback_captcha,
                                ],
                               layout={"display": "none"},
                               )

In [17]:
failed_captcha = widgets.Label("You excedeed the maximum number of trials.",
                               layout={"display": "none"}
                               )

In [18]:
timeout_captcha = 60 # / seconds
timeout_accept = 180 # / seconds
timeout_survey = 3000 # /seconds (for all similarity questions)
timeout_pair = timeout_survey / n_pairs # / seconds
timeout_experience = 120 # /seconds

message_timeout = widgets.Label("Sorry, your time expired!",
                                layout={"display": "none"},
                                )

In [19]:
file_conditions = Path(dir_info, "conditions.md")
conditions = make_markdown_widget(file_conditions)

accept = widgets.RadioButtons(options=["I Accept", "I Decline"], value="I Decline")


grid_initial = widgets.GridBox([conditions,
                                accept,
                                ],
                               layout={"display": "none"},
                               )

In [20]:
border_width = "2px"
border_style = "solid"
border_color = "black"
border_attributes = f"{border_width} {border_style} {border_color}"

layout_headers = dict(height="auto",
                      width="auto",
                      )

layout_pic = {"border": border_attributes}
pics = {suffix_mol: widgets.Image(format="svg+xml", layout=layout_pic) for suffix_mol in suffixes_mols}
header_pics = widgets.HTML("2D Representations:", layout=layout_headers)
grid_pics = widgets.AppLayout(left_sidebar=pics["a"],
                              right_sidebar=pics["b"],
                              )
grid_2d = widgets.GridBox([header_pics, grid_pics])

layout_view = {"border": border_attributes}
views = {suffix_mol: nv.NGLWidget(layout=layout_view) for suffix_mol in suffixes_mols}
grid_gap = "1%"
header_views = widgets.Label("3D Representations (interactive):", layout=layout_headers)
layout_reset_button = dict(width="auto")
reset_button = widgets.Button(description="Reset 3D Views", layout=layout_reset_button)
grid_views = widgets.AppLayout(left_sidebar=views["a"],
                               right_sidebar=views["b"],
                               footer=reset_button,
                               grid_gap=grid_gap,
                               #Setting pane heights prevents the formation of
                               #too much "empty space" below the reset button.
                               pane_heights=[0, 6, 1]
                               )
grid_3d = widgets.GridBox([header_views, grid_views])

file_question = Path(dir_info, "question_molecules.md")
question = make_markdown_widget(file_question)
categories_similarity = ["Yes", "No"]
similar = widgets.RadioButtons(options=categories_similarity,
                               value=None,
                               )
submit_str = "Submit Answer ({}/" + f"{n_pairs})"
submit_similar = widgets.Button()
epilog = widgets.Label("There are no right or wrong answers!")
feedback_similar = widgets.Output()
grid_similar = widgets.GridBox([question,
                                similar,
                                submit_similar,
                                epilog,
                                feedback_similar,
                                ],
                               )

#Select whether of 2D or 3D displays will be above for current user.
options_above = ["2D", "3D"]
random.seed(id_user)
above = random.choice(options_above)
if above == "2D":
    grid_above = grid_2d
    grid_below = grid_3d
elif above == "3D":
    grid_above = grid_3d
    grid_below = grid_2d
else:
    pass

grid_survey = widgets.GridBox([grid_similar,
                               grid_above,
                               grid_below,
                               ],
                               layout={"display": "none"},
                               )

In [21]:
question_experience = widgets.HTML("One last question: what is your current academic qualification?")

categories_experience = ["PhD Student",
                         "Postdoc",
                         "Professor / Researcher",
                         "None of the above",
                         ]
experience = widgets.RadioButtons(options=categories_experience,
                                  value=None,
                                  )

submit_experience = widgets.Button(description="Submit Answer")

feedback_experience = widgets.Output()


grid_experience = widgets.GridBox([question_experience,
                                   experience,
                                   submit_experience,
                                   feedback_experience,
                                   ],
                                  layout={"display": "none"},
                                  )

In [22]:
file_thanks = Path(dir_info, "thanks.md")
thanks = make_markdown_widget(file_thanks)

file_logo_group = Path(dir_info, "logo_group_Strasbourg.png")
logo_group = widgets.Image().from_file(file_logo_group)
logo_group.layout = {"max_width": "8cm"}

grid_thanks = widgets.GridBox([thanks,
                               logo_group,
                               ],
                              layout={"display": "none"}
                              )

In [23]:
def switch_widget(old, new):
    """Conceal `old` widget and show `new` widget."""
    old.layout.display = "none"
    new.layout.display = "block"

In [24]:
def commit_dispose(session, engine):
    """Commit database `session` and dispose of database `engine`.
Necessary to close connections to Heroku Postgres database, so that if user"""
    session.commit()
    engine.dispose()

In [25]:
def switch_commit_dispose(old, new, session, engine):
    """Conceal `old` widget and show `new` widget.
Also, commit current database `session` connection, and dispose of database `engine`.
This is necessary to close connections to Heroku Postgres database,
even if the window on which the app is launched is not closed."""
    switch_widget(old, new)
    commit_dispose(session, engine)

In [26]:
def new_displayed_molecule(mol: mda.Universe,
                           view: nv.NGLWidget,
                           ) -> nv.component.ComponentViewer:
    """Remove all existing molecules and display a new molecule in a NGLWidget view."""
    
    #Remove all existing components in `view`.
    #(if `view` is empty, no component will be removed.)
    for component in view:
        view.remove_component(component)
    
    #Add a new component, thus displaying `mol`.
    component = view.add_component(mol)
    
    #For a better display in Voilà.
    view.handle_resize()
    
    return component

In [27]:
def wait_for_click_captcha(field, submit_button, correct_value: str):
    future = asyncio.Future()
    
    counter_trials = Counter()
    max_trials = 3
    
    def on_button_clicked(counter_trials, submit_button):
        #The solution to count clicks on a Button is inspired
        #by: https://stackoverflow.com/questions/32616321/incrementing-a-counter-using-ipython-widgets
        #(!)It is not possible to use a simple integer counter, I have tried:
        #the count would remain 1.
        counter_trials.update("_")
        field_value = field.value
        
        if field_value == correct_value:
            success = True
            future.set_result(success)
            
            #Create database engine and session.
            #(!)It is important to create them after the captcha was passed,
            #otherwise each time an user connected to the Heroku app
            #(even a fake user who did not pass the captcha),
            #the Heroku Postgres database would count a connection,
            #and connections to Heroku database are limited!
            create_db_engine_and_session(db_objects)
            
#             #Add all tables to database.
#             Base.metadata.create_all(db_objects["engine"])
            
            #Add current User to table, updating its status.
            db_objects["session"].add(user)
            user.status = "Started"
            commit_dispose(db_objects["session"], db_objects["engine"])
        
        
        else:
            if counter_trials["_"] < max_trials:
                with feedback_captcha:
                    feedback_captcha.clear_output()
                    print("You submitted the wrong text! Try again!")
            else:
                success = False
                future.set_result(success)
        
        
    submit_button.on_click(partial(on_button_clicked, counter_trials))
    return future

In [28]:
def wait_for_change(widget, value, id_user: str):
    future = asyncio.Future()
    
    def getvalue(change):
        # make the new value available
        future.set_result(change.new)
        widget.unobserve(getvalue, value)
        
        #Update status of user,
        #and insert type of survey that will be displayed to him.
        user.status = "Accepted"
        user.display_above = above
        commit_dispose(db_objects["session"], db_objects["engine"])
        
        return change.new
    
    widget.observe(getvalue, value)
    return future

In [29]:
#Whether to synchronize the cameras of the 3D viewers
#for each molecular pair; change manually!
sync_cameras = False

In [30]:
def wait_for_click_similar(submit_button, id_pair, count_pair):
    future = asyncio.Future()
    
    #(!)Remember to display a widget before changing its value!
    #Otherwise, it will not work as intended (will not appear, will not update...).
    
    nums_atoms = []
    for suffix_mol in suffixes_mols:
        #Load 2D picture and display it.
        file_pic = Path(dir_pics, f"{bname_pics}_{id_pair:03}{suffix_mol}.svg")
        pics[suffix_mol].set_value_from_file(file_pic)
        
        #Load 3D conformer.
        file_mol = Path(dir_confs, f"{bname_confs}_{id_pair:03}{suffix_mol}.pdb")
        mol = mda.Universe(file_mol.as_posix())
        
        #Get number of atoms
        #(it will be used to define a default orientation).
        nums_atoms.append(mol.atoms.n_atoms)
            
        #Display molecule.
        new_displayed_molecule(mol, views[suffix_mol])
    
    
    #Calculate a default orientation for current molecular pair,
    #based on number of atoms.
    max_n_atoms = max(nums_atoms)
    default_orientation = [0 for i in range(15)]
    default_orientation += [1] #last element of 4x4 matrix is always 1
    value_diag = min(30, max_n_atoms)
    ids_diag = [0, 5, 10] #id of diagonal elements (except last)
    for i in ids_diag:
        default_orientation[i] = value_diag
    
    #And set default orientation to both molecules.
    views["a"]._set_camera_orientation(default_orientation)
    views["b"]._set_camera_orientation(default_orientation)
    
    
    if sync_cameras:
        #Make moving one molecule also move the other.
        views["a"]._set_sync_camera([views["b"]])
        views["b"]._set_sync_camera([views["a"]])
    else:
        pass
    
    
    #Change description of submit button, informing user
    #on the progress of the survey.
    submit_button.description = submit_str.format(count_pair)
    
    
    def reset_views(reset_button):
        views["a"]._set_camera_orientation(default_orientation)
        views["b"]._set_camera_orientation(default_orientation)
        
    
    reset_button.on_click(reset_views)
    
    
    def on_button_clicked(submit_button):
        
        answer = similar.value
        if answer == None:
            with feedback_similar:
                feedback_similar.clear_output()
                print("Please provide an answer to proceed")
        else:
            feedback_similar.clear_output()
            answer = Answer(id_pair=id_pair, similar=answer)
            future.set_result(answer)
    
    
    similar.value = None
    submit_button.on_click(on_button_clicked)
    return future

In [31]:
def wait_for_click_experience(submit_button):
    future = asyncio.Future()
    
    
    def on_button_clicked(submit_button):
        
        answer = experience.value
        if answer == None:
            with feedback_experience:
                feedback_experience.clear_output()
                print("Please provide an answer to complete the survey")
        else:
            feedback_experience.clear_output()
            future.set_result(answer)
        
    
    submit_button.on_click(on_button_clicked)
    return future

In [32]:
async def survey_molecules(ids_pairs) -> List[Answer]:
    answers = []
    for count_pair, id_pair in enumerate(ids_pairs, start=1):

        try:
            answer = await asyncio.wait_for(wait_for_click_similar(submit_similar, id_pair, count_pair),
                                            timeout=timeout_pair,
                                            )
            answers.append(answer)
        except asyncio.TimeoutError as error:
            raise asyncio.TimeoutError

    
    return answers

In [33]:
async def f():
    
    switch_widget(loading, grid_captcha)
    
    #Wait for user to complete captcha problem.
    try:
        success_captcha = await asyncio.wait_for(wait_for_click_captcha(field_captcha,
                                                                        submit_captcha,
                                                                        correct_value=value_captcha,
                                                                        ),
                                                 timeout=timeout_captcha,
                                                 )
        
        if success_captcha:
            switch_widget(grid_captcha, grid_initial)
        else:
            #Database Engine and Session were not created
            #if the user failed the captcha, so no need to commit and dispose.
            switch_widget(grid_captcha, failed_captcha)
            return "Failed Captcha"
            
    except asyncio.TimeoutError as error:
        #Database Engine and Session were not created
        #if the captcha timed out, so no need to commit and dispose.
        switch_widget(grid_captcha, message_timeout)
        return error
    
    
    #Wait for user to accept the terms and conditions.
    try:
        await asyncio.wait_for(wait_for_change(accept, "value", id_user),
                               timeout=timeout_accept
                               )
    
        switch_widget(grid_initial, grid_survey)
    
    except asyncio.TimeoutError as error:
        switch_commit_dispose(grid_initial,
                              message_timeout,
                              db_objects["session"],
                              db_objects["engine"],
                              )
        return error
    
    
    #Wait for user to answer all similarity questions,
    #creating answers DataFrame.    
    try:
        answers_similarity = await survey_molecules(ids_pairs)
        
        switch_widget(grid_survey, grid_experience)
        
    except asyncio.TimeoutError as error:
        switch_commit_dispose(grid_survey,
                              message_timeout,
                              db_objects["session"],
                              db_objects["engine"],
                              )
        return error
    
    
    #Wait for user to answer academic experience question,
    #and add to answers DataFrame.
    try:
        answer_experience = await asyncio.wait_for(wait_for_click_experience(submit_experience),
                                                   timeout=timeout_experience,
                                                   )


        #Add similarity answers and experience to current User, and update status.
        user.status = "Completed"
        user.answers = answers_similarity
        user.experience = answer_experience
        
        #Thank the user for completing survey.
        switch_commit_dispose(grid_experience,
                              grid_thanks,
                              db_objects["session"],
                              db_objects["engine"],
                              )
        
        return
    
    except asyncio.TimeoutError as error:
        switch_commit_dispose(grid_experience,
                              message_timeout,
                              db_objects["session"],
                              db_objects["engine"],
                              )
        return error


task = asyncio.create_task(f())

display(grid_captcha,
        failed_captcha,
        message_timeout,
        grid_initial,
        grid_survey,
        grid_experience,
        grid_thanks,
        )

GridBox(children=(Image(value=b'\x89PNG\r\n\x1a\n\x00\x00\x00\rIHDR\x00\x00\x01"\x00\x00\x00y\x08\x02\x00\x00\…

Label(value='You excedeed the maximum number of trials.', layout=Layout(display='none'))

Label(value='Sorry, your time expired!', layout=Layout(display='none'))

GridBox(children=(HTML(value='<h1>Molecular Similarity Survey</h1>\n<p>Many thanks for taking part in this res…

GridBox(children=(GridBox(children=(HTML(value='<h3>Do you think that the two molecules are similar?</h3>'), R…

GridBox(children=(HTML(value='One last question: what is your current academic qualification?'), RadioButtons(…

GridBox(children=(HTML(value='<h3>Thank you for taking the survey!</h3>\n<p>You can close the webpage.</p>'), …